## 2 - Data Embedding and Vectorization

This notebook is an example of how data can be split and embedded (using an embedding model) and then stored in a vector DB for use by RAG Systems

Components used:
* ChromaDB as the Vector Database Engine (only local in this notebook)
* SentenceTransformers for data embedding
* Langchain to glue everything together

In [ ]:
# install dependencies
!python --version
!pip install nltk matplotlib prettytable tqdm chromadb
!pip install langchain_community sentence_transformers
!pip install boto3 fastapi

In [ ]:
# global vars
TRAINING_DATA_PATH = "training_data/"

# import libraries
try:
    from langchain_core.documents import Document
    from langchain_community.document_loaders import DirectoryLoader, TextLoader
    from langchain_text_splitters import RecursiveCharacterTextSplitter as rts
    from prettytable import PrettyTable
    from chromadb.utils import embedding_functions as ef
    from chromadb import Client
    import uuid
    from tqdm.autonotebook import tqdm, trange
    from nltk import word_tokenize, sent_tokenize, download
    import os
    import yaml
    import boto3
    download("punkt")
    download("punkt_tab")
    import matplotlib.pyplot as plt
except Exception as e:
    print(f"Caught Exception {e}")

In [ ]:
# dictionary class that holds parameters
# load values from a yaml file
class Parameters(object):
    def __init__(self, data: dict):
        if type(data) != dict:
            raise TypeError(f"Parameters: expected 'dict', got {type(data)}.")
        else:
            self.data = data

        for k in self.data.keys():
            if type(self.data.get(k)) != dict:
                self.__setattr__(k, self.data.get(k))
            else:
                self.__setattr__(k, Parameters(self.data.get(k)))

# load parameters file and read values into a dictionary class
try:
    with open("parameters.yaml") as parms:
        config_parms = yaml.safe_load(parms)
    creds = Parameters(config_parms)
except yaml.YAMLError as e:
    print(f"Error loading YAML file: {e}")
    exit()
except Exception as e:
    print(f"Caught exception: {e}")
    exit()

In [ ]:
# download training data from s3 if needed
try:
    # connect to MinIO and prepare buckets
    print(f"Accessing S3 endpoint {creds.params.url} with ACCESS_KEY {creds.params.accessKey}...")

    # instantiate connection
    minio_api = boto3.client("s3", endpoint_url=creds.params.url, aws_access_key_id=creds.params.accessKey, aws_secret_access_key=creds.params.secretKey)
except Exception as e:
    print(f"Caught exception: {e}")

# create folder to store training data
os.makedirs(creds.training_data.path, exist_ok = True)

# get list of data files
try:
    data_files = minio_api.list_objects_v2(Bucket=creds.training_data.trainingDataBucket)
    if data_files.get("Contents"):
        for item in data_files.get("Contents"):
            minio_api.download_file(creds.training_data.trainingDataBucket,
                                    item.get('Key'),
                                    "/".join((creds.training_data.path, item.get('Key'))))
except Exception as e:
    print(f"Caught Exception {e}")

In [ ]:
# Text Processing Primitives
# Embedding function
def s_transformer(model: str = "all-MiniLM-L6-v2"):
    return ef.SentenceTransformerEmbeddingFunction(model_name=model)

# load text documents from filesystem
def load_text_documents(path: str = ".", pattern: str = "**/*.txt",
                        multithread: bool = False) -> list:
    loader = DirectoryLoader(path,
                             glob=pattern,
                             loader_cls=TextLoader,
                             loader_kwargs={'autodetect_encoding': True},
                             use_multithreading=multithread,
                             silent_errors=True,
                             show_progress=True)
    return loader.load()

# prepare the knowledge corpus for further processing
def prepare_corpus(raw_loader: list) -> list:
    # collect statistics in relation to the data corpus
    corpus = []
    for doc in raw_loader:
        document_tokens = word_tokenize(doc.page_content)
        document_sentences = sent_tokenize(doc.page_content)
        corpus.append({"metadata": doc.metadata,
                       "raw_sentences": document_sentences,
                       "sentence_count": len(document_sentences),
                       "wordcount": len(document_tokens),
                       "vocabulary": set(document_tokens),
                       "lexical_richness": len(set(document_tokens)) / len(document_tokens)})

    # display corpus
    data_table = PrettyTable()
    data_table.field_names = ["Dataset", "Word Count", "Sentence Count", "Vocabulary", "Lexical Richness"]
    for dataset in corpus:
        data_table.add_row([dataset.get("metadata").get("source"), dataset.get("wordcount"), dataset.get("sentence_count"), len(dataset.get("vocabulary")), dataset.get("lexical_richness")])

    # display dataset statistics
    print(data_table)

    # return processed data
    tokenized_data = []
    for doc in tqdm(corpus, ascii=True, desc="Tokenizing..."):
        metadata = doc.get("metadata")
        for data in doc.get("raw_sentences"):
            tokenized_data.append(Document(metadata=metadata, page_content=data))

    # return corpus and metadata
    return tokenized_data, corpus

# split corpus in chunks for vectorization
def split_text_documents(documents: list = None,
                         chunk_size: int = 1000,
                         chunk_overlap: int = 0) -> list:
    if documents is None:
        return None

    splitter = rts(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_documents(documents)

In [ ]:
# walk the raw data storage path and search for text documents
raw_loader = load_text_documents(TRAINING_DATA_PATH)

# collect statistics in relation to the data corpus
corpus, metadata = prepare_corpus(raw_loader)

In [ ]:
# plot dataset statistics graph
x_vals = [len(x.get("vocabulary")) for x in metadata]
y_vals = [100 * x.get("lexical_richness") for x in metadata]
y2_vals = [x.get("sentence_count") for x in metadata]

# plot vocabulary vs richness data
plt.subplot(211)
plt.ylabel("Lexical Richness")
plt.xlabel("Vocabulary")
plt.scatter(x_vals, y_vals)
# plot sentence count vs vocabulary
plt.subplot(212)
plt.ylabel("Sentence Count")
plt.xlabel("Vocabulary")
plt.scatter(x_vals, y2_vals)
plt.show()

In [ ]:
# prepare data for further tokenization
CHUNK_SIZE = 1000 # 100 words per chunk
OVERLAP = 10 # overlapping words

# split documents
tokenized_docs = split_text_documents(corpus, chunk_size=CHUNK_SIZE, chunk_overlap=OVERLAP)
print(f"Tokenized {len(tokenized_docs)} documents...")

# remove buffers
del(raw_loader)
del(corpus)
del(metadata)

In [ ]:
# now call the embedding model and upload data to the vector database
SIMILARITY_FUNCTION = "cosine"
COLLECTION_NAME = "rag_demo"
os.environ['HF_HOME'] = '/tmp/huggingface/hub/'

# connect to a running chroma instance
try:
    vector_store = chromadb.PersistentClient(path="/tmp")
    chroma_collection = vector_store.get_or_create_collection(COLLECTION_NAME,
                                                              metadata={"hnsw:space": SIMILARITY_FUNCTION},
                                                              embedding_function=s_transformer())
except Exception as e:
    print(f"Caught Exception: {e}")

In [ ]:
# embed data and push vectors to the database
if len(tokenized_docs) > 0:
    for doc in tqdm(tokenized_docs, ascii=True, desc="Ingesting..."):
        chroma_collection.add(ids=[str(uuid.uuid1())], documents=doc.page_content, metadatas=doc.metadata)

In [ ]:
# explore the vector collection
print(f"Objects stored in the collection {chroma_collection.name}: {chroma_collection.count()}")
chroma_collection.query(query_texts=["why openshift is better?"], n_results=2)